In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import mglearn
#from IPython.display import display
import warnings
warnings.filterwarnings("ignore")
from matplotlib import rc

In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [3]:
data= pd.read_csv("color_survey_answers.csv",  sep='\t')

In [4]:
data.head()


,id,user_id,datestamp,r,g,b,colorname
0,1,1,1.267419e+09,72,100,175,pastel blue
1,2,1,1.267419e+09,204,177,246,faint violet
2,3,1,1.267419e+09,182,226,245,baby blue
3,4,1,1.267419e+09,130,64,234,purple
4,5,2,1.267419e+09,75,49,234,blue


In [5]:
data.columns

Index(['id', 'user_id', 'datestamp', 'r', 'g', 'b', 'colorname'], dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3363127 entries, 0 to 3363126
Data columns (total 7 columns):
id           int64
user_id      int64
datestamp    float64
r            int64
g            int64
b            int64
colorname    object
dtypes: float64(1), int64(5), object(1)
memory usage: 179.6+ MB


In [7]:
#converting date from float to datetime
from datetime import datetime
data['datestamp'] = pd.to_datetime(data['datestamp'],unit='s')

In [8]:
# to check the null value in the dataset
print(data.isnull().values.sum())

1734


In [9]:
#data["colorname"].value_counts()

In [10]:
data = data.fillna({"colorname": "green"})

In [11]:
#extracting new colours into another dataframe from original

In [12]:
new_colour = data[data['colorname'] == 'impatiens juice'].copy()

In [13]:
new_colour = data[data['colorname'] == 'caribbean surf'].append(new_colour)

In [14]:
new_colour = data[data['colorname'] == 'medium tan leather'].append(new_colour)

In [15]:
new_colour = data[data['colorname'] == 'babypoo'].append(new_colour)

In [16]:
#new_colour.tail()

In [17]:
new_colour.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 185291 to 3170468
Data columns (total 7 columns):
id           8 non-null int64
user_id      8 non-null int64
datestamp    8 non-null datetime64[ns]
r            8 non-null int64
g            8 non-null int64
b            8 non-null int64
colorname    8 non-null object
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 512.0+ bytes


In [18]:
#increasing the occurance of new colours
new_colour = new_colour.append([new_colour]*20)

In [19]:
new_colour.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 185291 to 3170468
Data columns (total 7 columns):
id           168 non-null int64
user_id      168 non-null int64
datestamp    168 non-null datetime64[ns]
r            168 non-null int64
g            168 non-null int64
b            168 non-null int64
colorname    168 non-null object
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 10.5+ KB


In [20]:
#combining new colours to the filtered original dataset
data = pd.concat([data,new_colour],ignore_index=True)

In [21]:
#data['colorname'].value_counts()

In [22]:
#shuffling the dataset to have equal distribution of each colour in the dataset
data= data.sample(frac = 1,random_state= 10).reset_index(drop = True)

In [23]:
a= data['colorname'].value_counts()
b=dict(a)
c=list(b.keys())

In [24]:
N=100
data2=data[data.colorname.isin(c[:N])]
print(data2.shape)

(421, 7)


# on_hot_encoding

In [25]:
data_dummies=pd.get_dummies(data2, columns=["colorname"])
print(data_dummies.shape) 

(421, 106)


In [26]:
data_dummies.head()

,id,user_id,datestamp,r,g,b,colorname_#249ef7,colorname_#b76081,colorname_#f3f918,colorname_571a57,...,colorname_terrible blue,colorname_tic-tac green,colorname_turqouise emerald,colorname_ugly green/yellow,colorname_venusian vagina,colorname_yellow pages yellow,colorname_yellow-geen,colorname_yellowish vomit,colorname_you just lost the game green that color is sort of an inside joke actually.,colorname_yurtle the turtle green
2219,1949725,88436,2010-03-14 00:02:29,231,43,230,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11085,2312803,104293,2010-03-18 01:46:33,189,251,94,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20754,2403149,108128,2010-03-19 06:00:04,146,179,231,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21979,99729,5644,2010-03-01 15:37:33,30,179,38,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25341,2105988,95482,2010-03-15 22:09:12,46,117,51,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Split the data into features and target label
y = data_dummies.iloc[:,6:].values
X = data_dummies.iloc[:,3:6].values

In [28]:
X.shape

(421, 3)

In [29]:
y.shape

(421, 100)

In [30]:
# train and test splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0)

In [31]:
X_train.shape

(282, 3)

In [32]:
y_train.shape

(282, 100)

In [33]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [34]:
print("Test_Accuracy: {:.2f}".format(clf.score(X_test, y_test)))
print("Train_Accuracy: {:.2f}".format(clf.score(X_train, y_train)))

Test_Accuracy: 0.50
Train_Accuracy: 1.00


In [35]:
f=np.array([102,226,245]).reshape(1,-1)
color_list=list(data_dummies.columns[6:].values)

In [36]:
pred = clf.predict(f)

In [37]:
idx= pred.tolist()[0].index(1,0)

In [38]:
#print(colors_list[idx])

# custom binary encoding

In [39]:
## Again taking a subset of quite large dataset
#small_data_1 = data.sample(1000,random_state= 10)

In [40]:
data['color_green'] = np.where(data['colorname'].str.contains('green'), 1, 0)

In [41]:
data['color_red'] = np.where(data['colorname'].str.contains('red'), 1, 0)

In [42]:
data['color_pink'] = np.where(data['colorname'].str.contains('pink'), 1, 0)

In [43]:
data['color_blue'] = np.where(data['colorname'].str.contains('blue'), 1, 0)

In [44]:
data['color_orange'] = np.where(data['colorname'].str.contains('orange'), 1, 0)

In [45]:
data['color_yellow'] = np.where(data['colorname'].str.contains('yellow'), 1, 0)

In [46]:
data['color_purple'] = np.where(data['colorname'].str.contains('purple'), 1, 0)

In [47]:
data['color_tan'] = np.where(data['colorname'].str.contains('tan'), 1, 0)

In [48]:
data.head()

,id,user_id,datestamp,r,g,b,colorname,color_green,color_red,color_pink,color_blue,color_orange,color_yellow,color_purple,color_tan
0,2102379,95339,2010-03-15 21:30:48,99,23,111,purple,0,0,0,0,0,0,1,0
1,4130,166,2010-03-01 06:31:11,191,81,74,salmony-pink,0,0,1,0,0,0,0,0
2,2644842,118740,2010-03-22 18:33:06,198,128,109,red tan,0,1,0,0,0,0,0,1
3,2405676,108250,2010-03-19 06:33:03,55,34,6,black,0,0,0,0,0,0,0,0
4,2384434,107334,2010-03-19 02:44:38,166,132,201,joy''s headband,0,0,0,0,0,0,0,0


In [49]:
data.columns

Index(['id', 'user_id', 'datestamp', 'r', 'g', 'b', 'colorname', 'color_green',
       'color_red', 'color_pink', 'color_blue', 'color_orange', 'color_yellow',
       'color_purple', 'color_tan'],
      dtype='object')

In [50]:
data.shape

(3363295, 15)

In [51]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

In [52]:
# Extract all string-type columns
cols_str = []
for col in data:
    if is_string_dtype(data[col]):
        cols_str.append(col)
print(cols_str)

['colorname']


In [53]:
#make all string type variables to categorical variables.
for col in data:
    if is_string_dtype(data[col]):
        data[col] = data[col].astype('category').cat.as_ordered()

In [54]:
#Convert categorical variables to their numberic representations.
for col in data:
    if str(data[col].dtype) == "category":
        data[col] = data[col].cat.codes + 1

In [55]:
#small_data_1.iloc[:,6]

In [56]:
a= data['colorname'].value_counts()
b=dict(a)
c=list(b.keys())

In [57]:
N=100
data3=data[data.colorname.isin(c[:N])]
print(data3.shape)

(207, 15)


In [58]:
data3.head()

,id,user_id,datestamp,r,g,b,colorname,color_green,color_red,color_pink,color_blue,color_orange,color_yellow,color_purple,color_tan
1398,343221,16435,2010-03-02 06:20:54,64,142,235,74,0,0,0,1,0,0,0,0
4172,1759209,79929,2010-03-11 19:24:47,23,29,187,74,0,0,0,1,0,0,0,0
23333,401681,18512,2010-03-02 16:06:32,255,64,237,39,0,0,0,0,0,0,0,0
37740,2059723,93454,2010-03-15 13:24:24,179,173,132,66,0,0,0,0,0,0,0,0
59180,2610653,117160,2010-03-22 07:11:46,20,89,201,74,0,0,0,1,0,0,0,0


In [59]:
# Split the data into features and target label
y1 = data3.iloc[:,7:].values
X1 = data3.iloc[:,3:6].values

#y = np.array(sub_data['colorname'])
#X = np.array(sub_data.drop(['colorname', 'datestamp'], axis = 1))

In [60]:
# train and test splitting the data
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.33, random_state = 0)

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X1_train, y1_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [61]:
print("Test_Accuracy: {:.2f}".format(clf.score(X1_test, y1_test)))
print("Train_Accuracy: {:.2f}".format(clf.score(X1_train, y1_train)))

Test_Accuracy: 0.61
Train_Accuracy: 1.00


In [62]:
f=np.array([102,226,245]).reshape(1,-1)
color_list=list(data3.columns[7:].values)
pred = clf.predict(f)

In [63]:
idx= pred.tolist()[0].index(1,0)
#print(colors_list[idx])

Custom Binary shows better accuracy than on hot encoding